## Image Classification for Pneumonia via X-rays

In [ ]:
# Resources
# This one uses CNN: https://www.kaggle.com/code/homayoonkhadivi/medical-diagnosis-with-cnn-transfer-learning
# The codealong with CNN
#

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' 
import tensorflow as tf
from keras import models
from keras import layers
import pathlib
import PIL
import seaborn as sns
import time
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from sklearn.model_selection import train_test_split, cross_val_score
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping

import os
import sys


## About the Data:
The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care. 

Due to the relatively small amount of validation data (16 images), I will create my own validation data instead of using it, but I will still load it in.

In [ ]:
training = 'chest_xray/train'
testing = 'chest_xray/test'
validation = 'chest_xray/val'

In [ ]:
folder = 'chest_xray/train/NORMAL/'
path = folder

p = os.listdir(path)
pf = pd.DataFrame(p)
norm_weight = len(pf) / 5863

print(f' There are {len(pf[0])} images in the training:normal folder')

In [ ]:
folder = 'chest_xray/train/PNEUMONIA/'
path = folder
p = os.listdir(path)
pf = pd.DataFrame(p)
pneum_weight = len(pf) / 5863
pf
print(f' There are {len(pf[0])} images in the training:pneumonia folder')

In [ ]:
#From Codealong CNN
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'chest_xray/test', 
        target_size=(64, 64), batch_size = 624) 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'chest_xray/val', 
        target_size=(64, 64), batch_size = 16)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        'chest_xray/train', 
        target_size=(64, 64), batch_size=5216)

In [ ]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [ ]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

In [ ]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

## Baseline Model

In [ ]:
# Build a baseline fully connected model
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

baseline = model.fit(train_img,train_y,epochs=50, batch_size=32, 
                    validation_split= 0.1, steps_per_epoch= 25)


train_loss = baseline.history['loss']
train_acc = baseline.history['accuracy']
val_loss = baseline.history['val_loss']
val_acc = baseline.history['val_accuracy']

#Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=baseline.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=baseline.epoch, y=train_acc, ax=ax2, label='train_accuracy')

In [ ]:
# Build function that builds the model so we can evaluate in sklearn
def build_model():
    model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
    model.add(layers.Dense(7, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
keras_model = scikit_learn.KerasClassifier(build_model,
                                          epochs=50,
                                          steps_per_epoch= 10,
                                          batch_size=32,
                                          verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model, train_img, train_y, cv=3)

In [ ]:
results_train = model.evaluate(train_img, train_y)

In [ ]:
results_test = model.evaluate(test_img, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
# TODO: Add Confusion Matrix Code/ CF, as well as SaveResult

## Convolutional Neural Network (CNN)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch=25,
                    batch_size=32,  
                    validation_split=0.1)

In [ ]:
results_train = model.evaluate(train_images, train_y)

In [ ]:
results_test = model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history.history['loss']
sigmoid_acc = history.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

## CNN #1 Results



In [ ]:
def build_cnn():
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64 ,64,  3)))  
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model

In [ ]:
keras_model2 = scikit_learn.KerasClassifier(build_cnn,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model2, train_img, train_y, cv=3)

# CNN Model 2

In [ ]:
model2 = models.Sequential()

model2.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(96, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
history2 = model2.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model2.evaluate(train_images, train_y)

In [ ]:
results_test = model2.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history2.history['loss']
sigmoid_acc = history2.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history2.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history2.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

In [ ]:
def build_cnn2():
    model2.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))
    
    model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Flatten())
    model2.add(layers.Dense(64, activation='relu'))
    model2.add(layers.Dense(1, activation='sigmoid'))

    model2.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model2

In [ ]:
keras_model3 = scikit_learn.KerasClassifier(build_cnn2,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model3, train_img, train_y, cv=3)

# CNN Model 3

For this model, I will add batch normalization and dropout.

In [ ]:
model3 = models.Sequential()

model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
model3.add(BatchNormalization())
model3.add(layers.MaxPooling2D((2, 2)))



model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
model3.add(BatchNormalization())
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(Dropout(0.15))
model3.add(layers.Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model3.summary()

In [ ]:
history3 = model3.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model3.evaluate(train_images, train_y)

In [ ]:
results_test = model3.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history3.history['loss']
sigmoid_acc = history3.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history3.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history3.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

In [ ]:
def build_cnn3():
    model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))



    model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Flatten())
    model3.add(layers.Dense(64, activation='relu'))
    model3.add(Dropout(0.15))
    model3.add(layers.Dense(1, activation='sigmoid'))


    model3.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model3

In [ ]:
keras_model3 = scikit_learn.KerasClassifier(build_cnn3,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model3, train_img, train_y, cv=3)

# CNN Model 4

For this model, I will change the optimizer to adam

In [ ]:
model4 = models.Sequential()

model4.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
model4.add(BatchNormalization())
model4.add(layers.MaxPooling2D((2, 2)))


model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(96, (3, 3), activation='relu'))
model4.add(BatchNormalization())
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(Dropout(0.15))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model4.summary()

In [ ]:
history4 = model4.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model4.evaluate(train_images, train_y)

In [ ]:
results_test = model4.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history4.history['loss']
sigmoid_acc = history4.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history4.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history4.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

In [ ]:
def build_cnn4():
    model4.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model4.add(BatchNormalization())
    model4.add(layers.MaxPooling2D((2, 2)))


    model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model4.add(BatchNormalization())
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Flatten())
    model4.add(layers.Dense(64, activation='relu'))
    model4.add(Dropout(0.15))
    model4.add(layers.Dense(1, activation='sigmoid'))


    model4.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model4

In [ ]:
keras_model4 = scikit_learn.KerasClassifier(build_cnn4,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model4, train_img, train_y, cv=3)

# CNN Model 5

For this model, I will add weights and steps per epoch, with fewer epochs

In [ ]:
norm_weight

In [ ]:
pneum_weight

In [ ]:
classWeight = {0:norm_weight, 1:pneum_weight}

In [ ]:
model5 = models.Sequential()

model5.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(32, (4, 4), activation='relu'))
model5.add(BatchNormalization())
model5.add(layers.MaxPooling2D((2, 2)))


model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(96, (3, 3), activation='relu'))
model5.add(BatchNormalization())
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(64, activation='relu'))
model5.add(Dropout(0.15))
model5.add(layers.Dense(1, activation='sigmoid'))

model5.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model5.summary()

In [ ]:
history5 = model5.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch= 100,
                    batch_size=32,
                    validation_split= .1,
                    class_weight=classWeight)

In [ ]:
results_train = model5.evaluate(train_images, train_y)

In [ ]:
results_test = model5.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history5.history['loss']
sigmoid_acc = history5.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history5.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history5.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

In [ ]:
def build_cnn5():
    model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))


    model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Flatten())
    model3.add(layers.Dense(64, activation='relu'))
    model3.add(Dropout(0.15))
    model3.add(layers.Dense(1, activation='sigmoid'))


    model3.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model3

In [ ]:
keras_model5 = scikit_learn.KerasClassifier(build_cnn5,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2,
                                            class_weight=classWeight)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model4, train_img, train_y, cv=3)

# CNN Model 6

For this model, I will add dropout. 

In [ ]:
model6 = models.Sequential()

model6.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(32, (4, 4), activation='relu'))
model6.add(BatchNormalization())
model6.add(Dropout(0.15))
model6.add(layers.MaxPooling2D((2, 2)))


model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(Dropout(0.10))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(96, (3, 3), activation='relu'))
model6.add(BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(64, activation='relu'))
model6.add(Dropout(0.15))
model6.add(layers.Dense(1, activation='sigmoid'))

model6.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model6.summary()

In [ ]:
history6 = model6.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch= 100,
                    batch_size=32,
                    validation_split= .1,
                    class_weight=classWeight)

In [ ]:
results_train = model6.evaluate(train_images, train_y)

In [ ]:
results_test = model6.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
sigmoid_loss = history6.history['loss']
sigmoid_acc = history6.history['accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
sns.lineplot(x=history6.epoch, y = sigmoid_loss, ax = ax1, label = 'loss')
sns.lineplot(x=history6.epoch, y = sigmoid_acc, ax = ax2, label = 'accuracy')

In [ ]:
def build_cnn6():
    model6.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model6.add(BatchNormalization())
    model6.add(layers.MaxPooling2D((2, 2)))


    model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model6.add(BatchNormalization())
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Flatten())
    model6.add(layers.Dense(64, activation='relu'))
    model6.add(Dropout(0.15))
    model6.add(layers.Dense(1, activation='sigmoid'))

    model6.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model6

In [ ]:
keras_model6 = scikit_learn.KerasClassifier(build_cnn6,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2,
                                            class_weight=classWeight)

# Part 2: Re-splitting the data

I am in an odd spot with this data - my validation scores are matching up with my training scores, but my testing scores are not. 

Because I did not split this data myself, I think there may be an issue with the split. So, I will add together all the data (the train and test sections provided) and re-split said data. 

In [ ]:
os.mkdir('re-split_data')

In [ ]:

path = 're-split_data/NORMAL/'

p = os.listdir(path)
pf = pd.DataFrame(p)
len(pf[0])

In [ ]:
path = 're-split_data/PNEUMONIA/'

p = os.listdir(path)
pf = pd.DataFrame(p)
len(pf[0])

## Split

In [ ]:
X = 're-split_data/NORMAL'
y = 're-split_data/PNEUMONIA'


In [ ]:
#From Codealong CNN
# get all the data in the directory split/test (180 images), and reshape them
X_generator2 = ImageDataGenerator().flow_from_directory('re-split_data/NORMAL', target_size=(64, 64), batch_size = 1576) 


# get all the data in the directory split/train (542 images), and reshape them
y_generator2 = ImageDataGenerator().flow_from_directory('re-split_data/PNEUMONIA', target_size=(64, 64), batch_size=4221)

In [ ]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [ ]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

In [ ]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))